In [ ]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #


import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tqdm import tqdm

import xtrack as xt
import xobjects as xo
import xpart as xp


beta_rel = 0.305
#beta_rel = 0.106

gamma = 1.050
#gamma = 1.006


current=2.4
length = 1.5 # m cooler length
r_beam=25*1e-3

mass0=938.27208816*1e6 #ev/c^2

T_perp = 100e-3 # <E> [eV] = kb*T
T_l =  1e-3 # <E> [eV]
magnetic_field = 0.060 # T for AD
B_ratio=1e-10



c=299792458.0

p0c = mass0*beta_rel*gamma #eV/c



circumference = 182.43280000000 #m
T = circumference/(c*beta_rel)
s_per_turn = T


beta_x=10 
beta_y=4

#disp_x=0.12

                                        


Parameters: $I=2.4 A, B=0.060 T, \beta_x=10 m,\beta_y=4,D_x=0.12 m T_\perp=100meV,T_\parallel=1meV,\epsilon_{x}=35*1e-6,\epsilon_{y}=35*1e-6,dp/p=1e-3$

In [ ]:
num_particles = 1
delta_single = 5e-4
num_steps=50
#20*20*20 = 18 h
for sign in [-1, 1]:
    delta = sign * delta_single

    disp_values = np.linspace(-20, 20, num=num_steps)
    offset_x_values = np.linspace(-5e-4, 5e-4, num=num_steps)  # example delta values, adjust as needed
    offset_dp_values = np.linspace(-1e-9, 1e-9, num=num_steps)  # example delta values, adjust as needed

    action_reduction = np.zeros((len(disp_values), len(offset_x_values), len(offset_dp_values)))  # initialize empty array to store cooling_time values

    for i, disp in enumerate(tqdm(disp_values)):
        for j, offset_x in enumerate(offset_x_values):
            for k, offset_dp in enumerate(offset_dp_values):
                disp_x = disp

                arc = xt.LinearTransferMatrix(Q_x=5.44, Q_y=5.42,
                                    beta_x_0=beta_x, beta_x_1=beta_x,
                                    beta_y_0=beta_y,  beta_y_1=beta_y,
                                    alpha_x_0=0,   alpha_x_1=0,
                                    alpha_y_0=0,   alpha_y_1=0,
                                    disp_x_0=disp_x,disp_x_1=disp_x,
                                    disp_y_0=0,    disp_y_1=0,
                                    beta_s=1*1e40,
                                    Q_s=0,
                                    chroma_x=0.0, chroma_y=0)

                particles = xp.Particles(
                        mass0=mass0,
                        p0c=p0c,
                        x=disp_x*delta,
                        px=0,
                        y=0,
                        py=0,
                        delta=delta,
                        zeta=0)

                cooler = xt.ElectronCooler(current=current,length=length,r_beam=r_beam,
                                                        T_perp=T_perp,T_l=T_l,
                                                        magnetic_field=magnetic_field,B_ratio=1e-4,
                                                        Neutralisation_space_charge=1,
                                                        offset_dp=offset_dp, offset_x=offset_x)

                num_turns = int(1*1e3)
                #step = int(num_turns/100)

                
                action_x=[]
                # loop over turns
                for turn in tqdm(range(num_turns), desc='Cooling Progress', leave=False):
                    action_x_temp = (particles.x ** 2 / beta_x + beta_x * particles.px ** 2)
                    action_x.append(action_x_temp)

                    # track particle
                    arc.track(particles)
                    cooler.track(particles)

                action_reduction_temp=action_x[-1]/action_x[0]
                action_reduction[i, j, k] = action_reduction_temp  # store cooling_time value in the corresponding array position
    if sign == 1:
        np.savez('action_reduction_data_4d_pos.npz', action_reduction=action_reduction, offset_x_values=offset_x_values, offset_dp_values=offset_dp_values, disp_values=disp_values)
    else:
        np.savez('action_reduction_data_4d_neg.npz', action_reduction=action_reduction, offset_x_values=offset_x_values, offset_dp_values=offset_dp_values, disp_values=disp_values)
    
